### Imports

In [1]:
import pandas as pd
import numpy as np
import iexfinance as iex  #pip install iexfinance
from iexfinance.stocks import Stock
from datetime import datetime

import requests
import random
import time
from bs4 import BeautifulSoup
import re
from tqdm import tqdm

In [5]:
%load_ext autoreload
%autoreload 2

## Content overview for workflow: 
#### Stock data from IEX or some other source? (https://www.quora.com/Using-Python-whats-the-best-way-to-get-stock-data)

1. Load in stock data (how much?)
    - Find a list/source for tickers, so we know which stocks to fetch -> https://iextrading.com/trading/eligible-symbols/
    - Get individual stocks from certain indicies? Or random subset from the ticker list?
    - Maybe get different industry indicies?
        - We need some form of sector/industry segmentation for the stocks
        - We need open, close, industry, sector, market cap ... ?
1. Basic summary statistics
    - Which stocks, how many in each segment, countries?,
    - Calculate returns... 
    - generate new variables that we need (may only be daily returns)
1. Plots for individual stocks
    - What does e.g. the price of  Apple, Danske Bank look like over time?
1. Interactive plot
    - A plot that shows weighted returns for our given subsample
    - Slider with time period (start, end)
    - Segmentation selection


### Remember to comment on code :)

## Data load-in (Waldemar?)

This sections loads in historical daily price data for stocks in the US using the *IEX* python module.

The goal is to calculate daily returns (simple and continuously compounded (cc)) for each stock - and visualize this information in the subsequent sections.

In [3]:
x=iex.refdata.get_symbols() #list of dicts with ticker symbols, names of all stocks in IEX database

df_ticker=pd.DataFrame(x) #To a dataframe
df_ticker.sample(5) #view of 5  random stocks

,date,iexId,isEnabled,name,symbol,type
3576,2019-03-18,3097,True,Hallmark Financial Services Inc.,HALL,cs
4222,2019-03-18,8250,True,Intra-Cellular Therapies Inc.,ITCI,cs
2658,2019-03-18,2349,True,iShares MSCI United Kingdom,EWU,N/A
2993,2019-03-18,2589,True,First Trust Mid Cap Value AlphaDEX Fund,FNK,et
2605,2019-03-18,2304,True,Eaton Vance Senior Income Trust,EVF,cs


In [4]:
df_ticker.loc[df_ticker['name'] == 'Apple Inc.']

,date,iexId,isEnabled,name,symbol,type
13,2019-03-18,11,True,Apple Inc.,AAPL,cs


In [5]:
tickers= list(df_ticker['symbol']) # tickers for 100 random stocks #.loc[df_ticker['isEnabled']==True] .sample(100, random_state=10)
tickers[0:5]

['A', 'AA', 'AAAU', 'AABA', 'AAC']

In [6]:
#Initing different containers for the first ticker

sector={} #empty dict for sectors for each ticker

stock=Stock(tickers[0])
sect=stock.get_book()['quote']['sector']
sector.update({tickers[0] : sect})

df_close=pd.DataFrame(iex.stocks.get_historical_data(tickers[0], start='2015-01-02', end='2019-01-02', output_format='pandas')['close'])
df_close=df_close.rename(columns = {'close' : tickers[0]})

In [7]:
remove_stocks=[] #container for stocks that have tickers, but do not have historical data available

for i in tqdm(tickers[1:], leave=False): #runtime is ~3-4 hours?
    
    #Retrieving sector for each ticker
    try: 
        stock=Stock(i) #turning ticker i into iex.Stock object
        sect=stock.get_book()['quote']['sector'] #extracting sector from books
        sector.update({i : sect}) #appending to dict
    except:
        remove_stocks.append(i)
        
        
    #Creating df with close prices for each ticker
    try: 
        test=iex.stocks.get_historical_data(i, start='2015-01-02', end='2019-01-02', output_format='pandas')['close'] #returns a series
        test_df=pd.DataFrame(test)
        test_df=test_df.rename(columns = {'close' : i})
        
        #Final df with close prices with stock as a col
        df_close = pd.merge(df_close, test_df, how='left', left_index=True, right_index=True)
    except:
        remove_stocks.append(i)



  6%|████▍                                                                        | 496/8714 [11:29<2:55:07,  1.28s/it]

ARNC- does not exist


  9%|███████                                                                      | 801/8714 [18:38<3:12:16,  1.46s/it]

BFL does not exist


 15%|███████████▊                                                                | 1348/8714 [31:10<2:45:19,  1.35s/it]

...

VENUSDT does not exist


100%|███████████████████████████████████████████████████████████████████████████▉| 8712/8714 [3:31:02<00:02,  1.02s/it]

XLMUSDT does not exist


100%|███████████████████████████████████████████████████████████████████████████▉| 8713/8714 [3:31:03<00:01,  1.03s/it]

QTUMUSDT does not exist


In [8]:
#Turning SECTOR dict into series then dataframe:
x=pd.Series(sector)
df_sect=pd.DataFrame(x)
df_sect=df_sect.rename(columns={0 : 'sector'}) #renaming col

#Merging onto df_ticker
df_ticker=pd.merge(df_ticker, df_sect, how='left',  left_on=df_ticker['symbol'], right_index=True)

In [19]:
df_ticker.head()

,date,iexId,isEnabled,name,symbol,type,sector
0,2019-03-18,2,True,Agilent Technologies Inc.,A,cs,Healthcare
1,2019-03-18,12042,True,Alcoa Corporation,AA,cs,Basic Materials
2,2019-03-18,14924,True,Perth Mint Physical Gold,AAAU,NaN,NaN
3,2019-03-18,7653,True,Altaba Inc.,AABA,cs,Financial Services
4,2019-03-18,9169,True,AAC Holdings Inc.,AAC,cs,Healthcare


In [16]:
#exporting
#df_close.to_csv('df_close.csv', sep=';', index=True)
#df_ticker.to_csv('df_ticker.csv', sep=';', index=True)

## Data manipulation

In [13]:
df_close=pd.read_csv('df_close.csv', sep=';', index_col=0)
df_ticker=pd.read_csv('df_ticker.csv', sep=';', index_col=0)

In [14]:
#Simple returns dataframe
df_simple=df_close.copy().pct_change(1)

In [15]:
df_simple.head()

,A,AA,AAAU,AABA,AAC,AADR,AAL,AAMC,AAME,AAN,...,ZUO,ZVV,ZVVV,ZVZZT,ZWZZT,ZXIET,ZXZZT,ZYME,ZYNE,ZYXI
date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-05,-0.018737,NaN,NaN,-0.020730,-0.015889,-0.012228,-0.000649,0.041842,-0.004963,-0.003920,...,NaN,0.0,NaN,0.003996,0.641791,NaN,-0.933088,NaN,NaN,-0.111667
2015-01-06,-0.015579,NaN,NaN,0.001628,-0.011773,-0.008358,-0.015499,-0.037756,-0.022421,-0.037379,...,NaN,0.0,NaN,-0.442289,-0.119091,NaN,9990.208791,NaN,NaN,0.000000
2015-01-07,0.013273,NaN,NaN,-0.012599,-0.025187,0.000000,-0.000565,0.006128,0.000000,0.030313,...,NaN,0.0,NaN,0.072257,-0.276574,NaN,-0.000121,NaN,NaN,-0.062539
2015-01-08,0.029975,NaN,NaN,0.033752,0.025489,0.015622,0.012263,0.038908,0.007654,0.016198,...,NaN,0.0,NaN,0.046589,0.000000,NaN,-0.998108,NaN,NaN,0.000000


In [16]:
# cc returns dataframe
df_cc=df_simple.copy()
df_cc +=1
df_cc=df_cc.copy().apply(np.log)

In [17]:
df_cc.shape #days and number of stocks

(1007, 8683)

## Summary statistics (Mazzi?)

#### Simple daily returns and cc daily returns
simple returns: $R_t=\frac{P_t}{P_{t-1}}-1$

cc returns: $r_t=log\left(\frac{P_t}{P_{t-1}}\right)$

Whick implies: $r_t=log(1+R_t)$


Plot cc vs. simple to illustrate

## Descriptive plots (Sofie?)

## Interactive plot (Sofie + Waldemar?)

Need some installs: https://github.com/matplotlib/jupyter-matplotlib

`pip install ipympl`

`# If using JupyterLab`

`# Install nodejs: https://nodejs.org/en/download/`

`jupyter labextension install @jupyter-widgets/jupyterlab-manager`

`jupyter labextension install jupyter-matplotlib`


#### Content
1. Interactive plot with close price (from df_close) with slider for date, dropdown for ticker (multiple stock selection?)
1. Interactive plot with returns (either from df_simple or df_cc) with slider for date, dropdown for ticker
1. Interactive plot with indicies for sectors (need to create df) with slider for date, dropdown for sector (multiple selection allowed) ? (WALDEMAR)

In [23]:
import matplotlib.pyplot as plt
%matplotlib widget

#plt.figure()
#x = [1,2,3]
#y = [4,5,6]
#plt.plot(x,y)


plt.plot(df_close.sample(1, axis=1)) #Plotting random stock's closing price

FigureCanvasNbAgg()

### 